A Natural Language Processing project that aims to classify twitter sentiments as negative or positive.

In [2]:
#importing libraries
import pandas as pd
import numpy as np
import nltk


ModuleNotFoundError: No module named 'nltk.stopwords'